In [1]:
_debug=False

# Import Data

In [2]:
# Change working directory
import os
cwd = os.getcwd ()
fd = 'C:/Py/PyProj/2022Env387MIT808/GitHub/MIT-808/data/raw/' #ToDo how to make this generic to GitHub folders structure?
os.chdir(fd)
print(cwd, 'to', os.getcwd ())

C:\Py\PyProj\2022_Notebooks to C:\Py\PyProj\2022Env387MIT808\GitHub\MIT-808\data\raw


In [3]:
# Read source Excel
import pandas as pd
fn = 'query-sdg-full_20220312_1550.csv'
# Description and instrumenttag
df = pd.read_csv(fn, sep='|', header=0)

In [4]:
df.head(2)

,ptr_id,articletype_id,sdg_lst,authors,title,content,keywords,institution_id
0,13,13.0,"14, 2","{""Jansen, Jonathan D.""}",Autonomy and accountability in the regulation ...,This article examines the struggles of the Sou...,"{Autonomy,Learning,Performance,Teaching,""Gover...",1
1,14,14.0,2,"{""Jansen, Jonathan D.""}",Intellectuals under fire,Looks at the status of intellectuals in South ...,"{""Cultural policy"",Democracy,""Political system...",1


In [5]:
n=1
print(df.title[n])
print(df.content[n])
print('\n')
print(df.keywords[n])

Intellectuals under fire
Looks at the status of intellectuals in South Africa ten years after democracy was established in the country. Direction of democracy in the country; Concerns about the policies of the government and its response to criticism from intellectuals.


{"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy",Democracy,"Political systems",Intellectuals,Politics,"Cultural policy

# Evaluate Data Quality

In [6]:
# Unique ID
df.ptr_id.is_unique

True

In [7]:
# Unique ID
df.articletype_id.is_unique

True

In [8]:
#Report nulls
null_columns=df.columns[df.isnull().any()]
print('Columns with nulls:\n', df[null_columns].isnull().sum())
for col in null_columns:
    df[col]=df[col].fillna(0)

Columns with nulls:
 articletype_id     1
sdg_lst            1
content           23
dtype: int64


In [9]:
null_columns=df.columns[df.isnull().any()]
print('Columns with nulls:\n', df[null_columns].isnull().sum())

Columns with nulls:
 Series([], dtype: float64)


In [10]:
#check for NaN
df1 = df[df.isna().any(axis=1)]
df1.shape
#del df1

(0, 8)

In [11]:
# Check sum: ptr_id should equal articletype_id
print('count mismatchees:', df.loc[ (df.ptr_id != df.articletype_id)].shape)
print(df.loc[ (df.ptr_id != df.articletype_id)])
if _debug:
    print('shape', df.shape)
    print('drop', df.drop(df[ (df.ptr_id != df.articletype_id) ].index, inplace = True))
df.drop(df[ (df.ptr_id != df.articletype_id) ].index, inplace = True)

count mismatchees: (1, 8)
        ptr_id  articletype_id sdg_lst      authors             title  \
208806  342219             0.0       0  {"齋藤, 久美子"}  黨錮と學問 : とくに何休の場合   

                                                  content keywords  \
208806  黒人奴隷制に基づくタバコ・プランテーションの展開をみたメリーランド植民地とヴァジニア植民地は...   {NULL}   

        institution_id  
208806              38  


In [12]:
df.shape

(208806, 8)

### Clean up keywords feature to have unique phrases.

In [13]:
kw=df.keywords

In [14]:
if _debug:
    print(kw[0])

In [15]:
# strip left of '{'
left=kw.str.split('{').str[1]
# strip right of '}'
right=left.str.split('}').str[0]
kw=right.str.replace('"', "")

In [16]:
if _debug:
    print(kw[0])

In [17]:
# Convert delimited strings into lists; and keep unique strings. 
list_all = kw.str.split(',')
list_unique = []
for i, v in enumerate(list_all):
    list_unique.append(list(set(list_all[i])))   

In [18]:
if _debug:
    n=1
    print(list_unique[n])
    print(list_all[n])
    print(len(list_unique))

In [19]:
df['unique_keywords']=list_unique
del kw, list_all, list_unique

In [20]:
df.head(1).T

,0
ptr_id,13
articletype_id,13.0
sdg_lst,"14, 2"
authors,"{""Jansen, Jonathan D.""}"
title,Autonomy and accountability in the regulation ...
content,This article examines the struggles of the Sou...
keywords,"{Autonomy,Learning,Performance,Teaching,""Gover..."
institution_id,1
unique_keywords,"[Teaching, Performance, Learning, Educational ..."


### Convert sdg text into lists

In [21]:
sdg=df.sdg_lst

In [22]:
# list of strings
list_all = sdg.str.split(',')

#ToDo better code needed here
# list of integers 
n=0
list_int = []
for i, v1 in enumerate(list_all):
    lst = []
    
    try:
        for j, v2 in enumerate(list_all[i]):
            lst.append(int(v2))
        n=+1
    except:
        print('skip: ', i, n, list_all[i])
    
    list_int.append(lst)  

if _debug:
    print(list_all[0])
    print(list_int[0])
    
#append to df
df['sdg_ints']=list_int
del sdg, list_all, list_int

In [23]:
df

,ptr_id,articletype_id,sdg_lst,authors,title,content,keywords,institution_id,unique_keywords,sdg_ints
0,13,13.0,"14, 2","{""Jansen, Jonathan D.""}",Autonomy and accountability in the regulation ...,This article examines the struggles of the Sou...,"{Autonomy,Learning,Performance,Teaching,""Gover...",1,"[Teaching, Performance, Learning, Educational ...","[14, 2]"
1,14,14.0,2,"{""Jansen, Jonathan D.""}",Intellectuals under fire,Looks at the status of intellectuals in South ...,"{""Cultural policy"",Democracy,""Political system...",1,"[Political systems, Politics, Democracy, Cultu...",[2]
2,16,16.0,2,"{""Jansen, Jonathan D.""}",Mode 2 knowledge and institutional life: takin...,This paper examines the response of a black un...,"{""Higher education"",""Information technology"",""...",1,"[Information technology, Knowledge theory, App...",[2]
3,17,17.0,"14, 1","{""Jansen, Jonathan D.""}",Political symbolism as policy craft : explaini...,The policy literature in developing countries ...,"{Apartheid,""Educational change"",""Educational p...",1,"[Apartheid, Educational change, Theories, Educ...","[14, 1]"
4,18,18.0,14,"{""Jansen, Jonathan D.""}",School curriculum since apartheid : intersecti...,In the wake of South Africa's first non-racial...,"{""Curriculum development"",""Educational policy""...",1,"[Curriculum development, Apartheid, Educationa...",[14]
...,...,...,...,...,...,...,...,...,...,...
208801,229328,229328.0,2,"{""Amin, Kemi"",""Taylor, Anneliese S""}",Elevator Pitch Exercise Template and Examples,Use this elevator pitch template for exercises...,"{communications,library,bioinformatician,""scho...",34,"[open access, scholarly communication, bioinfo...",[2]
208802,229329,229329.0,1,"{""Yang, Su-Fen"",""Arnold, Barry C""}",A simple approach for monitoring business serv...,Control charts are effective tools for signal ...,"{Models,Statistical,Algorithms,Time,Commerce,""...",34,"[Taiwan, Commerce, Algorithms, Quality Control...",[1]
208803,229330,229330.0,16,"{""Stein, Hans Henrik"",""Casas, Gloria Amparo"",""...",Nutritional value of high fiber co-products fr...,High fiber co-products from the copra and palm...,"{""Copra expellers"",""Copra meal"",""Palm kernel e...",34,"[Copra expellers, Rice bran, Animal Production...",[16]
208804,229331,229331.0,"1, 7, 8","{""Bromfield, Heather"",""Moore, Eli""}",Unfair Shares: Racial Disparities and the Regi...,This report reveals that in the San Francisco ...,"{""Bay Area allocations"",methods,terminology,pe...",34,"[income, Bay Area allocations, methods, housin...","[1, 7, 8]"
